In [1]:
import os
import re
import time
import random
import asyncio
import warnings
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor,  as_completed
import pandas as pd
import nest_asyncio
import cloudscraper
from htmldate import find_date
from bs4 import BeautifulSoup
from newspaper import Article

warnings.filterwarnings('ignore')

In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

MAX_WORKERS = 10
MAX_RETRIES = 5

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    ),
    "Accept": (
        "text/html,application/xhtml+xml,application/xml;"
        "q=0.9,image/avif,image/webp,*/*;q=0.8"
    ),
    "Referer": "https://www.investing.com/",
}

scraper = cloudscraper.create_scraper(
    browser={'browser': 'chrome', 'platform': 'windows'}
)

def fetch_page(page: int):
    url = "https://www.investing.com/news/most-popular-news"
    if page > 1:
        url += f"/{page}"
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = scraper.get(url, headers=HEADERS, timeout=30)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "lxml")

            anchors = soup.select(
                'ul[data-test="news-list"] '
                'li article a[data-test="article-title-link"]'
            )
            return [a["href"] for a in anchors if a.has_attr("href")]
        except Exception as e:
            if attempt < MAX_RETRIES:
                backoff = 2 ** (attempt - 1) + random.random()
                time.sleep(backoff)
            else:
                print(f"Page {page} failed after {MAX_RETRIES}: {e}")
    return []

def robust_scrape(max_pages=10):
    first = fetch_page(1)
    PER_PAGE = len(first)
    if PER_PAGE == 0:
        raise RuntimeError("Failed to fetch the first page. Please check headers or cookies.")

    print(f"Fetched {PER_PAGE} links on page 1")

    results = {1: first}

    if max_pages == 1 or PER_PAGE == 0:
        print("Only one page detected or no pagination found.")
        return first

    pages = list(range(2, max_pages + 1))

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
        futures = {pool.submit(fetch_page, p): p for p in pages}
        for fut in as_completed(futures):
            p = futures[fut]
            results[p] = fut.result()

        for round in range(1, MAX_RETRIES + 1):
            bad = [p for p, links in results.items() if len(links) != PER_PAGE]
            if not bad:
                print(f"All pages OK after {round - 1} retries")
                break
            print(f"Retry round {round} for pages: {bad}")
            futures = {pool.submit(fetch_page, p): p for p in bad}
            for fut in as_completed(futures):
                p = futures[fut]
                results[p] = fut.result()
        else:
            print("Retry limit reached; some pages may still be incomplete.")

    total_fetched = sum(len(links) for links in results.values())
    expected = PER_PAGE * max_pages
    print(f"Total links fetched (including duplicates): {total_fetched} (expected {expected})")

    all_links = set(link for links in results.values() for link in links)
    print(f"Final: got {len(all_links)} unique URLs")
    return list(all_links)

if __name__ == "__main__":
    links = robust_scrape(max_pages=1)


Fetched 20 links on page 1
Only one page detected or no pagination found.


In [3]:
nest_asyncio.apply()

FETCH_WORKERS = min(32, os.cpu_count() * 4)  
PROCESS_WORKERS = os.cpu_count() or 4
MAX_FETCH_RETRIES = 3                      
RETRY_DELAY = 1                             
scraper = cloudscraper.create_scraper()

def is_placeholder(html: str) -> bool:
    lower = html.lower() if html else ""
    return (
        'temporarily down for maintenance' in lower
        or 'just a moment' in lower
        or "we're temporarily down" in lower
    )

def safe_find_datetime(url, html_content=None):
    try:
        dt = find_date(url)
        if dt:
            return dt, "00:00"
    except:
        pass
    
    if html_content:
        m = re.search(r"(\d{1,2}/\d{1,2}/\d{4}),\s*(\d{1,2}:\d{2}\s*(?:AM|PM))", html_content)
        if m:
            ds, ts = m.groups()
            try:
                dt = datetime.strptime(f"{ds}, {ts}", "%m/%d/%Y, %I:%M %p")
                return dt.strftime("%Y-%m-%d"), dt.strftime("%H:%M")
            except:
                pass
        
        m = re.search(r"(\d{2}/\d{2}/\d{4}),\s*(\d{2}:\d{2})", html_content)
        if m:
            ds, ts = m.groups()
            for fmt in ("%d/%m/%Y, %H:%M", "%m/%d/%Y, %H:%M"):
                try:
                    dt = datetime.strptime(f"{ds}, {ts}", fmt)
                    return dt.strftime("%Y-%m-%d"), dt.strftime("%H:%M")
                except:
                    continue
    
    now = datetime.now()
    return now.strftime("%Y-%m-%d"), now.strftime("%H:%M")

def fetch_html(url, idx, total):
    for attempt in range(1, MAX_FETCH_RETRIES + 1):
        try:
            resp = scraper.get(url, timeout=30)
            html = resp.text
            if is_placeholder(html):
                raise RuntimeError('Placeholder')
                
            print(f"[Fetch][{idx}/{total}][ok]")
            return url, html
            
        except Exception:
            print(f"[Fetch][{idx}/{total}][retry {attempt}]")
            if attempt < MAX_FETCH_RETRIES:
                time.sleep(RETRY_DELAY)
                
    print(f"[Fetch error] {idx}/{total}: failed after {MAX_FETCH_RETRIES} retries")
    return url, None

def process_article(arg):
    url, html = arg
    if not html:
        return None
        
    art = Article(url)
    art.set_html(html)
    
    try:
        art.parse()
    except:
        return None
        
    text = art.text or ""
    title = (art.title or "").strip() or "No title"
    
    date, tm = safe_find_datetime(url, html)
    
    return {'publish_date': date, 'publish_time': tm,
             'title': title, 'body_text': text, 'url': url}

async def scrape_all(urls):
    total = len(urls)
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(max_workers=FETCH_WORKERS) as fetch_pool:
        fetch_tasks = [loop.run_in_executor(fetch_pool, fetch_html, u, i+1, total)
                       for i, u in enumerate(urls)]
        fetched = await asyncio.gather(*fetch_tasks)

    records = []
    with ThreadPoolExecutor(max_workers=PROCESS_WORKERS) as proc_pool:
        futures = {
            proc_pool.submit(process_article, fr): fr[0]
            for fr in fetched if fr[1]
        }
        
        for i, fut in enumerate(as_completed(futures), 1):
            res = fut.result()
            print(f"[Process][{i}/{total}] {futures[fut]}")
            if res:
                records.append(res)
            
    return pd.DataFrame(records)

def main(links):
    df = asyncio.get_event_loop().run_until_complete(scrape_all(links))
    return df

if __name__ == '__main__':
    df = main(links)

[Fetch][1/20][ok]
[Fetch][17/20][ok]
[Fetch][19/20][ok]
[Fetch][8/20][ok]
[Fetch][6/20][ok]
[Fetch][2/20][ok]
[Fetch][4/20][ok]
[Fetch][9/20][ok]
[Fetch][13/20][ok]
[Fetch][3/20][ok]
[Fetch][5/20][ok]
[Fetch][15/20][ok]
[Fetch][20/20][ok]
[Fetch][10/20][ok]
[Fetch][18/20][ok]
[Fetch][16/20][ok]
[Fetch][14/20][ok]
[Fetch][12/20][ok]
[Fetch][7/20][ok]
[Fetch][11/20][ok]
[Process][1/20] https://www.investing.com/news/politics-news/musk-calls-trumps-big-beautiful-bill-porkfilled-4078918
[Process][2/20] https://www.investing.com/news/stock-market-news/piper-sandler-is-tesla-the-only-us-automaker-with-a-plan-to-bypass-china-4077712
[Process][3/20] https://www.investing.com/news/economy-news/wolfe-trump-trump-budget-bill-slightly-worse-for-deficits-than-current-policy-4077823
[Process][4/20] https://www.investing.com/news/cryptocurrency-news/bitcoin-price-today-recovers-to-105k-but-trade-economic-jitters-limit-upside-4076970
[Process][5/20] https://www.investing.com/news/stock-market-news/nas

In [4]:
df=df.sort_values(by=['publish_date', 'publish_time'], ascending=[False,False]).reset_index(drop=True)
pd.set_option('display.max_columns', None)

In [5]:
empty_body_count = df[df['body_text'] == ''].shape[0]
print(f"Number of articles with empty body_text: {empty_body_count}")

Number of articles with empty body_text: 0


In [6]:
import os
from dotenv import load_dotenv

load_dotenv("../SentimentAnalysis/GPT/secret.env")

api_key = os.getenv("GEMINI_API_KEY")

if api_key is None:
    print("Error: GEMINI_API_KEY not found in .env file or environment variables.")
else:
    print("GEMINI_API_KEY loaded successfully.")

GEMINI_API_KEY loaded successfully.


In [7]:
import google.generativeai as genai
from tqdm import tqdm

genai.configure(api_key=api_key)
generation_config = genai.GenerationConfig(
        temperature=0,
)
model = genai.GenerativeModel("gemini-2.5-flash-preview-04-17", generation_config=generation_config)

In [8]:
from google.api_core import retry

def is_retryable(e) -> bool:
    if retry.if_transient_error(e):
        return True
    elif (isinstance(e, genai.errors.ClientError) and e.code == 429):
        return True
    elif (isinstance(e, genai.errors.ServerError) and e.code == 503):
        return True
    else:
        return False

@retry.Retry(predicate=is_retryable)
def generate_content_with_rate_limit(prompt):
  return model.generate_content(prompt).text

In [9]:
prompt = """You are a financial news analyst specializing in stock market impact. Your task is to analyze the provided news article, summarize its core content concisely, determine its sentiment (positive, negative, or neutral), and assess its importance to the specified stock.

Here is the news from stock [STOCK] title and body:
---
[TITLE]
---
[BODY]
---

Please provide your analysis in the following format (Don't forget to make space between the sections as shown):

**Sentiment:**
[Positive / Negative / Neutral]

**Summary:**
[Your concise summary of the article, typically 2-3 sentences.]

**Reasoning for Sentiment:**
[Brief explanation (1-2 sentences) of why you categorized the sentiment as such, referencing key points or tone from the article.]

**Importance to Stock [STOCK]:**
[1-5, where 1 is minimal importance and 5 is very high importance.Answer in 1-5 only, no explanation.] (Answer only in number 1-5)

**Reasoning for Importance:**
[Brief explanation (1-2 sentences) of why you assigned this importance score, referencing specific details from the article that would impact the stock.]"""

In [10]:
predicted = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Prompting"): 
    current_stock = row.get("ticker", "news")

    filled_prompt = prompt.replace("[STOCK]", current_stock)
    filled_prompt = filled_prompt.replace("[TITLE]", row["title"])
    filled_prompt = filled_prompt.replace("[BODY]", row["body_text"])

    try:
        response = generate_content_with_rate_limit(filled_prompt)
        finalprediction = response.strip()
        if not finalprediction:
            print(f"Row {index}: LLM returned an empty string.")
            predicted.append("LLM_EMPTY_RESPONSE")
        else:
            predicted.append(finalprediction)
    except ValueError as ve:
        print(f"Row {index}: ValueError - {ve}. Appending 'ERROR_VALUE_ERROR'.")
        predicted.append("ERROR_VALUE_ERROR")
        continue
    except Exception as e:
        if "429 Too Many Requests" in str(e) or "quota" in str(e).lower():
            print(f"Row {index}: Rate Limit Exceeded or Quota Error - {e}. Appending 'ERROR_RATE_LIMIT'.")
            predicted.append("ERROR_RATE_LIMIT")
        elif "safety" in str(e).lower() or "blocked" in str(e).lower():
             print(f"Row {index}: Content Safety/Blocked - {e}. Appending 'ERROR_SAFETY_BLOCKED'.")
             predicted.append("ERROR_SAFETY_BLOCKED")
        else:
            print(f"Row {index}: Unexpected Error - {e}. Appending 'ERROR_UNEXPECTED'.")
            predicted.append("ERROR_UNEXPECTED")
        continue

Prompting: 100%|██████████| 20/20 [02:07<00:00,  6.38s/it]


In [11]:
import numpy as np
predicted = np.array(predicted)
df["predicted"] = predicted

In [12]:
df["sentiment"] = df["predicted"].apply(lambda x: x.split("\n")[1].strip() if len(x.split("\n")) > 1 else None)
df["importance"] = df["predicted"].apply(lambda x: x.split("\n")[10].strip() if len(x.split("\n")) > 10 else None)
df["summary"] = df["predicted"].apply(lambda x: x.split("\n")[4].strip() if len(x.split("\n")) > 4 else None)

In [13]:
df = df[df['sentiment'].isin(['Positive', 'Negative', 'Neutral'])]
df = df[df['importance'].isin(['1', '2', '3', '4', '5'])]

In [14]:
model = genai.GenerativeModel("gemini-2.0-flash-001", generation_config=generation_config)
prompt = """Translate the following English sentence to Thai. Do not translate proper nouns, company names, product names, abbreviations, or technical terms — keep them in English. Do not provide any explanation, just the translation.
[TEXT]"""

In [15]:
translate = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Prompting"): 

    filled_prompt = prompt.replace("[TEXT]", row["summary"])

    try:
        response = generate_content_with_rate_limit(filled_prompt)
        finalprediction = response.strip()
        if not finalprediction:
            print(f"Row {index}: LLM returned an empty string.")
            translate.append("LLM_EMPTY_RESPONSE")
        else:
            translate.append(finalprediction)
    except ValueError as ve:
        print(f"Row {index}: ValueError - {ve}. Appending 'ERROR_VALUE_ERROR'.")
        translate.append("ERROR_VALUE_ERROR")
        continue
    except Exception as e:
        if "429 Too Many Requests" in str(e) or "quota" in str(e).lower():
            print(f"Row {index}: Rate Limit Exceeded or Quota Error - {e}. Appending 'ERROR_RATE_LIMIT'.")
            translate.append("ERROR_RATE_LIMIT")
        elif "safety" in str(e).lower() or "blocked" in str(e).lower():
             print(f"Row {index}: Content Safety/Blocked - {e}. Appending 'ERROR_SAFETY_BLOCKED'.")
             translate.append("ERROR_SAFETY_BLOCKED")
        else:
            print(f"Row {index}: Unexpected Error - {e}. Appending 'ERROR_UNEXPECTED'.")
            translate.append("ERROR_UNEXPECTED")
        continue

Prompting: 100%|██████████| 20/20 [01:25<00:00,  4.26s/it]


In [16]:
TICKER="news"

In [17]:
import datetime
if 'predicted' in df.columns:
    df.drop(columns=['predicted'], inplace=True)
if 'body_text' in df.columns:
    df.drop(columns=['body_text'], inplace=True)
now = datetime.datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M").strip().replace(' ', '_')
filename = f"Gemini_{TICKER}_{date_time}.csv"

In [18]:
df["translate"] = translate
df.to_csv(f"Data/{filename}", index=False)

In [19]:
data_filepath_for_streamlit = os.path.join("..", "CompletePipeline", "Data", filename)

print(f"Path to be passed to Streamlit: {data_filepath_for_streamlit}")

!streamlit run ../Visualization/news_app.py "{data_filepath_for_streamlit}"

Path to be passed to Streamlit: ../CompletePipeline/Data/Gemini_news_2025-06-04_14-08.csv

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.16.0.2:8501

YF.download() has changed argument auto_adjust default to True
^C
